#This code is based off of code developed by David Shean (UW) for SnowEx HackWeek 2021. The goal of this code is to create a comprehensive approach to gather Alaska SNOTEL data from an updated website so that individual datasets do not need to be downloaded each time. This code removes the need to use the NRCS Report Generator: https://wcc.sc.egov.usda.gov/reportGenerator/

###This code was created by LANL summer intern Madeline Beck - MSc Student, Montana State University
####Code last updated August 2022

##Step 1: Provide the necessary URL and packages to run the remainder of this code.

In [ ]:
# Instead of having to create and download data from the NRCS page each time, this URL contains the most recent data available 
# for every SNOTEL in the United States.
# Using this URL, this code will download the most recent data versions for all SNOTEL stations in the US. 

# This is the latest CUAHSI API endpoint and should not need to be updated
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [ ]:
# Install the required libraries and packages
# Proxy issues may not allow this code to be run using a Jupyter Notebook. However, this code runs correctly in Google Colab
%%capture
# %%capture hides most of the outputs, just make sure your code is correct!
# the -q makes it quiet (does not show all of the loading)
!pip install geopandas -q 
!pip install fiona -q
!pip install pyproj -q
!pip install ulmo -q 
!pip install geoviews -q
!pip install contextily -q
!pip install geoplot -q
!pip install folium

In [ ]:
## Import the necessary libaries and packages. 
## If any of the do not work you will need to install them in the above codeblock
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
from shapely.geometry import Point
import contextily as ctx
import ulmo
import fiona
from IPython.display import HTML
from pyproj import CRS
import contextily as cx
import folium
from geopandas import GeoDataFrame
import seaborn as sns

In [ ]:
# Capture all of the site data available using the data retrieval approach documented by Ulmo
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

##Step 2: Capture site locational data for each SNOTEL in the United States.

In [ ]:
# Create a dataframe from the site data just retrieved. 
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()
# Read in the head of the data to make sure that it has been uploaded correctly.
sites_df.head()

,code,name,network,location,elevation_m,site_property
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,"{'latitude': '41.2358283996582', 'longitude': ...",1886.7120361328125,"{'county': 'Modoc', 'state': 'California', 'si..."
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,"{'latitude': '37.522171020507813', 'longitude'...",2712.719970703125,"{'county': 'Kane', 'state': 'Utah', 'site_comm..."
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,"{'latitude': '45.59722900390625', 'longitude':...",2529.840087890625,"{'county': 'Madison', 'state': 'Montana', 'sit..."
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,"{'latitude': '61.749668121337891', 'longitude'...",48.768001556396484,"{'county': 'Matanuska-Susitna', 'state': 'Alas..."
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,"{'latitude': '47.779571533203125', 'longitude'...",1066.800048828125,"{'county': 'King', 'state': 'Washington', 'sit..."


##Clean up the DataFrame and prepare Point geometry objects
Convert 'location' column (contains dictionary with 'latitude' and 'longitude' values) to Shapely Point objects.
Store as a new 'geometry' column (needed by GeoPandas).
Drop the 'location' column, as this is no longer needed.
Update the dtype of the 'elevation_m' column to float

In [ ]:
# Use this code to create a new column based on the location data of each of the SNOTEL sites.
# Changing this to a geometry will allow for easy visualization within Python.
sites_df['geometry'] = np.array([Point(float(loc['longitude']), float(loc['latitude'])) for loc in sites_df['location']])
# Check the head of the data to make sure the geometry column has populated correctly. 
sites_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged 

,code,name,network,location,elevation_m,site_property,geometry
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,"{'latitude': '41.2358283996582', 'longitude': ...",1886.7120361328125,"{'county': 'Modoc', 'state': 'California', 'si...",POINT (-120.79192352294922 41.2358283996582)
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,"{'latitude': '37.522171020507813', 'longitude'...",2712.719970703125,"{'county': 'Kane', 'state': 'Utah', 'site_comm...",POINT (-112.27117919921875 37.52217102050781)
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,"{'latitude': '45.59722900390625', 'longitude':...",2529.840087890625,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (-111.95902252197266 45.59722900390625)
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,"{'latitude': '61.749668121337891', 'longitude'...",48.768001556396484,"{'county': 'Matanuska-Susitna', 'state': 'Alas...",POINT (-150.88966369628906 61.74966812133789)
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,"{'latitude': '47.779571533203125', 'longitude'...",1066.800048828125,"{'county': 'King', 'state': 'Washington', 'sit...",POINT (-121.69847106933594 47.779571533203125)


In [ ]:
# Continue to clean up the data by dropping the location column (we have the geometry) and cleaning up the elevation data.
sites_df = sites_df.drop(columns='location')
# First convert the elevation to a float.
sites_df = sites_df.astype({"elevation_m":float})
# And then we can convert to an integer.  
sites_df = sites_df.astype({"elevation_m":int})

To set the coordinate system, we need to first change the dataframe for sites into a geoDataFrame.

In [ ]:
# Set the sites to a GeoDataFrame to make sure that we can correctly change the CRS
sites_all = GeoDataFrame(sites_df)
# Define the coordinate reference system for all of the sites. 
sites_gdf_all = sites_all.set_crs(epsg=4326)
sites_gdf_all.head()

,code,name,network,elevation_m,site_property,geometry
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,1886,"{'county': 'Modoc', 'state': 'California', 'si...",POINT (-120.79192 41.23583)
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,2712,"{'county': 'Kane', 'state': 'Utah', 'site_comm...",POINT (-112.27118 37.52217)
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (-111.95902 45.59723)
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,48,"{'county': 'Matanuska-Susitna', 'state': 'Alas...",POINT (-150.88966 61.74967)
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,1066,"{'county': 'King', 'state': 'Washington', 'sit...",POINT (-121.69847 47.77957)


##Step 3: Filter for SNOTEL stations within Montana

In [ ]:
# Create a dataframe for all of the sites in Alaska
mt_sites = sites_gdf_all.loc[sites_gdf_all['code'].str.contains('MT')]
# Read in the data to make sure that each snotel is indeed in AK
mt_sites

,code,name,network,elevation_m,site_property,geometry
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (-111.95902 45.59723)
SNOTEL:307_MT_SNTL,307_MT_SNTL,Badger Pass,SNOTEL,2103,"{'county': 'Pondera', 'state': 'Montana', 'sit...",POINT (-113.02317 48.13088)
SNOTEL:311_MT_SNTL,311_MT_SNTL,Banfield Mountain,SNOTEL,1706,"{'county': 'Lincoln', 'state': 'Montana', 'sit...",POINT (-115.44573 48.57120)
SNOTEL:313_MT_SNTL,313_MT_SNTL,Barker Lakes,SNOTEL,2514,"{'county': 'Deer Lodge', 'state': 'Montana', '...",POINT (-113.13038 46.09713)
SNOTEL:315_MT_SNTL,315_MT_SNTL,Basin Creek,SNOTEL,2188,"{'county': 'Silver Bow', 'state': 'Montana', '...",POINT (-112.52047 45.79737)
...,...,...,...,...,...,...
SNOTEL:850_MT_SNTL,850_MT_SNTL,Warm Springs,SNOTEL,2377,"{'county': 'Granite', 'state': 'Montana', 'sit...",POINT (-113.16400 46.27368)
SNOTEL:924_MT_SNTL,924_MT_SNTL,West Yellowstone,SNOTEL,2042,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (-111.09199 44.65866)
SNOTEL:858_MT_SNTL,858_MT_SNTL,Whiskey Creek,SNOTEL,2072,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (-111.14998 44.61088)
SNOTEL:862_MT_SNTL,862_MT_SNTL,White Mill,SNOTEL,2651,"{'county': 'Park', 'state': 'Montana', 'site_c...",POINT (-109.90987 45.04575)


In [ ]:
# Set the CRS to a coordinate system appropriate for Alaska
mt_sites = mt_sites.to_crs(crs='EPSG:3604')
# Make sure that the geometry values have changed to represent the new CRS
mt_sites.head()

,code,name,network,elevation_m,site_property,geometry
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (408228.838 152738.800)
SNOTEL:307_MT_SNTL,307_MT_SNTL,Badger Pass,SNOTEL,2103,"{'county': 'Pondera', 'state': 'Montana', 'sit...",POINT (337945.391 437136.838)
SNOTEL:311_MT_SNTL,311_MT_SNTL,Banfield Mountain,SNOTEL,1706,"{'county': 'Lincoln', 'state': 'Montana', 'sit...",POINT (161741.985 496830.677)
SNOTEL:313_MT_SNTL,313_MT_SNTL,Barker Lakes,SNOTEL,2514,"{'county': 'Deer Lodge', 'state': 'Montana', '...",POINT (319506.824 211770.729)
SNOTEL:315_MT_SNTL,315_MT_SNTL,Basin Creek,SNOTEL,2188,"{'county': 'Silver Bow', 'state': 'Montana', '...",POINT (365320.406 176490.912)


## The current form of the data has the locational data saved as a geometry, but we want different columns for longitude and latitude so we can plot the data. 
### This code creates new columns for each SNOTEL location so we can export a new file to use in GIS.

In [ ]:
# Extract the geometry x term to a new column titled "X" for the Longitude
mt_sites['X'] = mt_sites['geometry'].x
# Extract the geometry y term to a new column titled "Y" for the Latitude
mt_sites['Y'] = mt_sites['geometry'].y
# Check to make sure the new columns have been updated with the correct data. 
mt_sites

,code,name,network,elevation_m,site_property,geometry,X,Y
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (408228.838 152738.800),408228.838415,152738.800346
SNOTEL:307_MT_SNTL,307_MT_SNTL,Badger Pass,SNOTEL,2103,"{'county': 'Pondera', 'state': 'Montana', 'sit...",POINT (337945.391 437136.838),337945.391391,437136.837740
SNOTEL:311_MT_SNTL,311_MT_SNTL,Banfield Mountain,SNOTEL,1706,"{'county': 'Lincoln', 'state': 'Montana', 'sit...",POINT (161741.985 496830.677),161741.984703,496830.676615
SNOTEL:313_MT_SNTL,313_MT_SNTL,Barker Lakes,SNOTEL,2514,"{'county': 'Deer Lodge', 'state': 'Montana', '...",POINT (319506.824 211770.729),319506.823531,211770.729201
SNOTEL:315_MT_SNTL,315_MT_SNTL,Basin Creek,SNOTEL,2188,"{'county': 'Silver Bow', 'state': 'Montana', '...",POINT (365320.406 176490.912),365320.405624,176490.911625
...,...,...,...,...,...,...,...,...
SNOTEL:850_MT_SNTL,850_MT_SNTL,Warm Springs,SNOTEL,2377,"{'county': 'Granite', 'state': 'Montana', 'sit...",POINT (317828.143 231484.958),317828.143449,231484.958281
SNOTEL:924_MT_SNTL,924_MT_SNTL,West Yellowstone,SNOTEL,2042,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (473714.444 46711.227),473714.443893,46711.226999
SNOTEL:858_MT_SNTL,858_MT_SNTL,Whiskey Creek,SNOTEL,2072,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (469003.068 41496.906),469003.068143,41496.906205
SNOTEL:862_MT_SNTL,862_MT_SNTL,White Mill,SNOTEL,2651,"{'county': 'Park', 'state': 'Montana', 'site_c...",POINT (567710.089 88533.058),567710.088899,88533.057696


## Export the created data. You can mount your drive to save the file directly to your drive. If not, the code as is will create an output file in the folder icon on the left of the screen which can be downloaded. 

In [ ]:
# Use the following code to export a .csv file that contains the locational data of all SNOTEL sites in Alaska.
mt_sites.to_csv('mt_sites_CRS.csv')

##Step 4: Further filter for the SNOTEL stations of interest for your research question.

### Specify the SNOTEL stations you would like to download data from. If you would like to add or subtract SNOTEL stations from the analysis, that can be done here. 

In [ ]:
# Create a array of the SNOTEL names that we want to grab from the mt_sites_all dataframe.
# We will use this array to filter through and obtain data for each of the SNOTEL stations.
# Sites can be added or removed from Chena_SNOTEL if you do not want ech of the seven to be used
Bridger_SNOTELs = ('Sacajawea','Brackett Creek')
# Create a dataframe to hold the sites of interest to us for our project
bridger_df = mt_sites[mt_sites['name'].isin(Bridger_SNOTELs)]
# Check the dataframe to make sure that each site has correctly been added
bridger_df

,code,name,network,elevation_m,site_property,geometry,X,Y
SNOTEL:365_MT_SNTL,365_MT_SNTL,Brackett Creek,SNOTEL,2231,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (488402.953 183400.660),488402.952914,183400.659851
SNOTEL:929_MT_SNTL,929_MT_SNTL,Sacajawea,SNOTEL,1996,"{'county': 'Gallatin', 'state': 'Montana', 'si...",POINT (489196.223 181483.703),489196.222880,181483.702970


In [ ]:
# Create a .csv that contains the locations of all the SNOTELs of interest.
bridger_df.to_csv('Bridger_SNOTELs.csv')

##Step 5: Download site data from the SNOTELs of interest

In [ ]:
#Get current datetime so we can download the most recent data
today = datetime.today().strftime('%Y-%m-%d')
today # Check to make sure it's correct

'2022-09-26'

In [ ]:
# Create variables for the start and end date so we can run the following code
# Set the start date to be in the past, likely before data is available to make sure that you capture all of the necessary data
start_date = datetime(1950,1,1)
end_date = today

## The following function uses Ulmo Cuahsi capabilities to set up the ability to download all the necessary site data for our SNOTEL datasets.

In [ ]:
ulmo.cuahsi.wof.get_variable_info(wsdlurl)

{'SNOTEL:BATN_D': {'value_type': 'Field Observation',
  'data_type': 'Minimum',
  'general_category': 'Soil',
  'sample_medium': 'Not Relevant',
  'no_data_value': '-9999',
  'speciation': 'Not Applicable',
  'code': 'BATN_D',
  'id': '2',
  'name': 'Battery voltage',
  'vocabulary': 'SNOTEL',
  'time': {'is_regular': True,
   'interval': '1',
   'units': {'abbreviation': 'd',
    'code': '104',
    'name': 'day',
    'type': 'Time'}},
  'units': {'abbreviation': 'V',
   'code': '168',
   'name': 'volts',
   'type': 'Potential Difference'}},
 'SNOTEL:BATT_D': {'value_type': 'Field Observation',
  'data_type': 'Continuous',
  'general_category': 'Soil',
  'sample_medium': 'Not Relevant',
  'no_data_value': '-9999',
  'speciation': 'Not Applicable',
  'code': 'BATT_D',
  'id': '3',
  'name': 'Battery voltage',
  'vocabulary': 'SNOTEL',
  'time': {'is_regular': True,
   'interval': '1',
   'units': {'abbreviation': 'd',
    'code': '104',
    'name': 'day',
    'type': 'Time'}},
  'units'

In [ ]:
# Define a function to pull data from the CUAHSI databse
def snotel_fetch(sitecode, variablecode, start_date=start_date, end_date=today):
    '''
    The following is a function to extract SNOTEL data from the CUAHSI website. 
    The code runs so that SNOTEL site data is downloaded from 1950-01-01, a date well before SNOTEL stations were installed to ensure entire period
    of record. The end date is set to whatever day this code is run, but the end date can be set the same way as start date in the block above.

    The code structure is set up so that multiple sitecodes and variable codes can be called.
    
    values_df = An empty dataframe to hold the values extracted as the function runs
    param sitecode = The corresponding sitecode of the SNOTEL station of interest
    param variablecode = The variable codes of the data you wish to capture
    param start_date = The first date to start searching for SNOTEL data records
    param end_date = Set to today by default, but can be changed in a different time record is desired
    '''
    # Create an empty dataframe to be populated in the function
    values_df = None
    # For the defined parameter specifications, try the following
    try:
        #Request data from the server
        site_values = ulmo.cuahsi.wof.get_values(wsdlurl, sitecode, variablecode, start=start_date, end=end_date)
        #Convert to a Pandas DataFrame   
        values_df = pd.DataFrame.from_dict(site_values['values'])
        #Parse the datetime values to Pandas Timestamp objects
        values_df['datetime'] = pd.to_datetime(values_df['date_time_utc'], utc=True)
        #Set the DataFrame index to the Timestamps
        values_df = values_df.set_index('datetime')
        #Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        #Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']
    # If the above does not work, print that the data is unable to be fetched    
    except:
        print("Unable to fetch %s" % variablecode)
    # Return the populated dataframe    
    return values_df

### Create a list that includes the sitecodes of all of the SNOTELs of interest for the Chena Watershed.

In [ ]:
# Make a function to get all sitecode values for whichever dataframe you would like to use
def sitecode_list_create(dataframe):
  '''
  A function to call a dataframe of interest and return all sitecode values contained for SNOTEL stations in that dataframe
  param dataframe = A dataframe to call to extract all site_code information from
  '''
  # Create an empty list to hold the site codes
  sitecode_list = list()
# Loop through and capture the sitecodes into a list
  for i in range(0,len(dataframe)):
    # Save the sitecodes for each row of the dataframe
    sitecode = dataframe.code[i]
    # Append this to the empty list
    sitecode_list.append(sitecode)
  # Return a list of the sitecodes of the station of interest  
  return sitecode_list

In [ ]:
# Create a list to hold the chena site values
bridger_list = sitecode_list_create(bridger_df)
# View to make sure it was done correctly
bridger_list

['365_MT_SNTL', '929_MT_SNTL']

###Create a list that includes all of the SNOTEL variables we would like to collect. The variables can be changed, but are based on current SNOTEL data downloads in the WRF-Hydro Assimilation folder.

In [ ]:
# Make a list for all of the variable codes
# We only need certain variables, so we do not want to write a loop like the above sitecodes.
# These variable codes can be changed if we want additional or less variables
# Currently, we are capturing daily precipitation, snow depth, average temperature, maximum temperature,
# minimum temperature, and snow water equivalent
variable_list = ['PRCPSA_D','SNWD_D','TAVG_D','TMAX_D','TMIN_D','WTEQ_D']

###Create a function to drop extraneous columns that we do not need for further analysis. 

In [ ]:
# Define a function to drop unneccesary columns within the data retrieval code below. 
# Create a list of columns that are no longer desired
columns_toDrop = ('censor_code', 'qualifiers', 'date_time_utc', 'method_id', 'method_code', 'source_code', 'quality_control_level_code')
def drop_cols(site_data, cols = columns_toDrop):
  '''
  A function to drop columns from a dataframe containing desired fetched SNOTEL variables
  param site_date = A called dataframe to drop extra data columns
  param cols = Predefined columns that are unnecesary for further analysis
  '''
  # For each column in the 'site_data' dataframe
  for column in site_data:
    # For the name of the column in the 'columns to drop' list
    for name in cols:
      # If the two column names match
      if column == name:
        # Drop the column
        site_data = site_data.drop(columns = column)
  # Return the site_data dataframe after the extra columns have been dropped      
  return site_data

##Use the snotel_fetch function to collect all of the variables of interest for each of the Chena SNOTELs for their entire period of record. Create a new dataframe for each of the SNOTEL stations of interest. 
###Note that this code can take a few minutes to run and sometimes will fail. If it fails, simply run again.

In [ ]:
# Run the function for each of the sites in our sites of interest for the Chena watershed
# Also have the function collect all of the information for the list of variables
# Create an empty dataframe to hold all the data
sitecode_df = pd.DataFrame([])
# Loop through the sitecode list and add to the dataframe
for i in range(0,len(bridger_list)):
  # Loop through the variable code list and add to the dataframe
  for j in range(0,len(variable_list)):
    # print the sitecode name
    print('Working on '+bridger_list[i] + ' ' + variable_list[j])
    # Save a new dataframe for each one the sitecodes
    df = snotel_fetch(bridger_list[i], variable_list[j])
    # Add a column with the sitecode info
    df['name'] = bridger_list[i]
    df['variable'] = variable_list[j]
    # Append the new dataframe to the empty dataframe
    sitecode_df = sitecode_df.append(df)
    sitecode_df = drop_cols(sitecode_df)
    # Create different dataframes for each of the stations
    Brackett = (sitecode_df.loc[sitecode_df['name'] == '365_MT_SNTL'])
    Sacajawea = (sitecode_df.loc[sitecode_df['name'] == '929_MT_SNTL'])

Working on 365_MT_SNTL PRCPSA_D
Working on 365_MT_SNTL SNWD_D
Working on 365_MT_SNTL TAVG_D
Working on 365_MT_SNTL TMAX_D
Working on 365_MT_SNTL TMIN_D
Working on 365_MT_SNTL WTEQ_D
Working on 929_MT_SNTL PRCPSA_D
Working on 929_MT_SNTL SNWD_D
Working on 929_MT_SNTL TAVG_D
Working on 929_MT_SNTL TMAX_D
Working on 929_MT_SNTL TMIN_D
Working on 929_MT_SNTL WTEQ_D


# This is the beginning of the final cleanup and data export! Pay attention and make sure to fill out the code correctly!

#START

###Here are the site names for the Chena River Watershed. Copy and paste these names in order to run the final code. 
Fairbanks, LChena, Monument, MtRyan, Munson, Teuchet, UChena

## Here are the names for the Willow Creek Watershed. Copy and paste these names in order to run the final code
independence

## Assign the name of the station you would like to export as the variable site_name

In [ ]:
# Assign the name of the SNOTEL site you would like to export
# If the name is changed correctly, the rest of the code should all run and produce a cleaned data export
site_name = Brackett
export_name = 'Brackett'
site_name

,value,name,variable
datetime,,,
1994-10-01 00:00:00+00:00,0.5,365_MT_SNTL,PRCPSA_D
1994-10-02 00:00:00+00:00,0.0,365_MT_SNTL,PRCPSA_D
1994-10-03 00:00:00+00:00,0.1,365_MT_SNTL,PRCPSA_D
1994-10-04 00:00:00+00:00,0.2,365_MT_SNTL,PRCPSA_D
1994-10-05 00:00:00+00:00,0.0,365_MT_SNTL,PRCPSA_D
...,...,...,...
2022-09-22 00:00:00+00:00,0.0,365_MT_SNTL,WTEQ_D
2022-09-23 00:00:00+00:00,0.0,365_MT_SNTL,WTEQ_D
2022-09-24 00:00:00+00:00,0.0,365_MT_SNTL,WTEQ_D


In [ ]:
# Create individual dataframes for when the variable code is equal to each of the descriptors
Precip = (site_name.loc[site_name['variable'] == 'PRCPSA_D'])
Depth = (site_name.loc[site_name['variable'] == 'SNWD_D'])
TAvg = (site_name.loc[site_name['variable'] == 'TAVG_D'])
TMax = (site_name.loc[site_name['variable'] == 'TMAX_D'])
TMin = (site_name.loc[site_name['variable'] == 'TMIN_D'])
WaterEQ = (site_name.loc[site_name['variable'] == 'WTEQ_D'])

# Change the column name to be more descriptive
Precip = Precip.rename(columns = {'value':'Daily Precipitation (in)'})
Depth = Depth.rename(columns = {'value':'Daily Snow Depth (in)'})
TAvg = TAvg.rename(columns = {'value':'Daily Average Temperature (F)'})
TMax = TMax.rename(columns = {'value':'Daily Maximum Temperature (F)'})
TMin = TMin.rename(columns = {'value':'Daily Minimum Temperature (F)'})
WaterEQ = WaterEQ.rename(columns = {'value':'Daily Water Equivalent (in)'})

In [ ]:
# Create a list with the final columns that we want to drop to clean the dataframe
final_dropCols = ('name', 'variable')
# Define the function to drop the columns for each of the individual dataframes
def final_cleanup(site_data, cols = final_dropCols):
  for column in site_data:
    for name in cols:
      if column == name:
        site_data = site_data.drop(columns = column)
  return site_data

In [ ]:
# Drop the extraneous columns for each of the dataframes
Precip = final_cleanup(Precip)
Depth = final_cleanup(Depth)
TAvg = final_cleanup(TAvg)
TMax = final_cleanup(TMax)
TMin = final_cleanup(TMin)
WaterEQ = final_cleanup(WaterEQ)

#Set the name of the station for the final export

In [ ]:
# Create a final dataframe for export that combines each of the individual dataframes on the datetime index
merged = pd.concat([Precip, Depth, TAvg, TMax, TMin, WaterEQ], axis = 1, join = 'outer')
# Check to make sure the data looks correct and that all column names have been updated
merged

,Daily Precipitation (in),Daily Snow Depth (in),Daily Average Temperature (F),Daily Maximum Temperature (F),Daily Minimum Temperature (F),Daily Water Equivalent (in)
datetime,,,,,,
1994-09-29 00:00:00+00:00,NaN,NaN,47.30,55.40,42.80,NaN
1994-09-30 00:00:00+00:00,NaN,NaN,46.04,53.06,40.82,NaN
1994-10-01 00:00:00+00:00,0.5,NaN,48.20,63.14,34.34,0.0
1994-10-02 00:00:00+00:00,0.0,NaN,27.86,36.14,19.22,0.0
1994-10-03 00:00:00+00:00,0.1,NaN,21.20,23.72,16.88,0.0
...,...,...,...,...,...,...
2022-09-22 00:00:00+00:00,1.7,0.0,42.44,55.22,35.96,0.0
2022-09-23 00:00:00+00:00,0.1,0.0,47.12,55.04,38.30,0.0
2022-09-24 00:00:00+00:00,0.1,0.0,50.54,57.20,46.40,0.0


In [ ]:
# Export to a csv file
# These files will contain values for each variable for the entirety of the period of record
merged.to_csv(export_name + '_export.csv')

## **FINISH**

##Cycle through the above code to create exports for all of the SNOTEL sites necessary